In [6]:
from datasets import load_dataset, DatasetDict, load_from_disk
# from huggingface_hub import snapshot_download


In [12]:
! pwd

/Users/aakarshrai/VSprojects/VFL/src


In [25]:
import os
import numpy as np
import pickle

tracks_dir = "../StarCraft-Motion/tracks"
for fname in os.listdir(tracks_dir):
    if not fname.endswith(".npz"):
        continue

    path = os.path.join(tracks_dir, fname)
    # load the archive
    shard = np.load(path, allow_pickle=True)

    # see what keys are inside
    print(shard.files)  
    # e.g. ["track", "metadata", "video"]  (might vary slightly)

    # pull them out
    track    = shard["track"]        # → np.ndarray, shape (T,N,18), dtype=uint8
    metadata = shard.keys()  # → Python dict
    # print(metadata)
    # print(track[0])
    # if there’s also a 'video' key, it’ll be raw bytes you can write out:
    # with open("tmp.mp4","wb") as f: f.write(shard["video"].tobytes())

    # now exactly like before:
    # print(fname, "→", track.shape, "frames;", metadata["game"]["map_name"])
    break
print("Track shape:", track.shape)
# print("track n:" track[0].shape[0])
for i in track[0]:
    print(i)


['track']
Track shape: (57, 206, 18)
[ 56  16  94  93 127   1 191 255   0   0 255   0   0   0   0   0   0   0]
[ 56  16  38  79 127   1 191 255   0   0 255   0   0   0   0   0   0   0]
[ 56  16  44  77 127   1 191 255   0   0 255   0   0   0   0   0   0   0]
[ 56  16  69  19 127   1 191 255   0   0 255   0   0   0   0   0   0   0]
[ 56  16  62 144 127   1 191 255   0   0 255   0   0   0   0   0   0   0]
[ 56  16 128  19 127   1 191 255   0   0 255   0   0   0   0   0   0   0]
[ 58  16  96  82 127   1 191 255   0   0 255   0   0   0   0   0   0   0]
[ 58  16  18 160 127   1 191 255   0   0 255   0   0   0   0   0   0   0]
[ 58  16  36  81 127   1 191 255   0   0 255   0   0   0   0   0   0   0]
[ 58  16  69 153 127   1 191 255   0   0 255   0   0   0   0   0   0   0]
[ 58  16  62 147 127   1 191 255   0   0 255   0   0   0   0   0   0   0]
[ 58  16  67  21 127   1 191 255   0   0 255   0   0   0   0   0   0   0]
[ 78  16  26 163 127   1 191 255   0   0 255   0   0   0   0   0   0   0]
[

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from concurrent.futures import ThreadPoolExecutor

# ----- Paths to StarCraft-Motion data -----
TRACKS_DIR = "../StarCraft-Motion/tracks"
# pick one shard or loop through many; for demo we use the first
shard_file = next(f for f in os.listdir(TRACKS_DIR) if f.endswith(".npz"))
npz_path = os.path.join(TRACKS_DIR, shard_file)

# ----- Load track tensor -----
shard = np.load(npz_path, allow_pickle=True)
track = shard["track"]            # shape (T, N, 18), uint8

# ----- Build dataset: flatten (T, N) into samples -----
samples = track.reshape(-1, 18).astype(np.float32)
# filter out destroyed units (status=5 at idx=5), if desired:
# samples = samples[samples[:,5] != 5]


# inputs and labels
y = torch.from_numpy(samples[:, 11].astype(np.int64))  # coarse goal type labels
X = torch.from_numpy(samples)
f_list = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]  # feature indices to keep
client_splits = [X[:, 0]]
client_splits.append([X[:, 1]])
client_splits.append([X[:, i] for i in range(2,5)])
for i in range(5, 11):
    client_splits.append(X[:, i])

# ----- Split features across two clients -----
# Client 1 sees state features [0:9), Client 2 sees [9:18)
feature_dims = [9, 9]
client_splits = torch.split(X, feature_dims, dim=1)
dataset = TensorDataset(*client_splits, y)
loader = DataLoader(dataset, batch_size=1024, shuffle=True)



In [27]:
!pip install torch

  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached MarkupSafe-3.0.2-cp313-cp313-macosx_11_0_arm64.whl.metadata (4.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 MB 7.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 8.1 MB/s eta 0:00:00a 0:00:01
Using cached jinja2-3.1.6-py3-none-any.whl (134 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 8.7 MB/s eta 0:00:00
Using cached MarkupSafe-3.0.2-cp313-cp313-macosx_11_0_arm64.whl (12 kB)
Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)

[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [28]:
%pip install torch


[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from concurrent.futures import ThreadPoolExecutor

# ----- Paths to StarCraft-Motion data -----
TRACKS_DIR = "../StarCraft-Motion/tracks"
shard_file = next(f for f in os.listdir(TRACKS_DIR) if f.endswith(".npz"))
npz_path = os.path.join(TRACKS_DIR, shard_file)
shard = np.load(npz_path, allow_pickle=True)
track = shard["track"]            # (T, N, 18), uint8

# ----- j -----
samples = track.reshape(-1, 18).astype(np.float32)
y = torch.from_numpy(samples[:, 11].astype(np.int64))  # coarse goal labels
X = torch.from_numpy(samples)

# Split features across clients
feature_dims = [9, 9]  # example: client 1 sees first 9 features, client 2 sees last 9
client_splits = torch.split(X, feature_dims, dim=1)
dataset = TensorDataset(*client_splits, y)
loader = DataLoader(dataset, batch_size=1024, shuffle=True)



In [ ]:
from torch.utils.data import IterableDataset, DataLoader, get_worker_info
import glob

class StarCraftDataset(IterableDataset):

    def __init__(self, tracks_dir):
        self.files = glob.glob(os.path.join(tracks_dir, "*.npz"))
        # define feature groups (state features indices 0-10)
        self.client_idx_groups = [
            [0],    # unit_type
            [1],    # unit_owner
            [2,3,4],# pos.x, pos.y, pos.z
            [5],    # status
            [6],    # facing
            [7],    # health
            [8],    # shield
            [9],    # energy
            [10],   # build_progress
        ]

    def __iter__(self):
        worker_info = get_worker_info()
        if (get_worker_info() is None):
            files = self.files
        else:
            per_worker = int(np.ceil(len(self.files) / float(worker_info.num_workers)))
            start = worker_info.id * per_worker
            end = min(start + per_worker, len(self.files))
            files = self.files[start:end]
        
        for file in files:
            sequences = []
            shard = np.load(file, allow_pickle = True)
            track = shard["track"]
            track = torch.from_numpy(track[:, : , :11].astype(np.float32))  # keep only first 11 features
            for n in range(track.shape[1]):
                sample = track[:, n, :].astype(np.float32)
                for i in range(track.shape[0] - self.seq_len):
                    unit = sample[i:i + self.seq_len]
                    label = unit[-1, 2:5]
                    client_feats = [unit[:, idx] for idx in self.client_idx_groups]
                    sequences.append((client_feats, label))

            for seq in sequences:
                yield seq



In [ ]:
TRACKS_DIR = "../StarCraft-Motion/tracks"
dataset = StarCraftDataset(TRACKS_DIR, seq_len=8)
loader = DataLoader(dataset,
                    batch_size=1024,
                    shuffle=False,
                    num_workers=4)

In [ ]:
# ----- Transformer-based models -----
# You can use PyTorch's built-in Transformer modules:
#   nn.TransformerEncoderLayer & nn.TransformerEncoder

def build_transformer(input_dim: int, embed_dim: int, n_heads: int, n_layers: int):
    """
    Creates a Transformer encoder stack for tabular features.
    We project features into token embeddings, then apply Transformer layers.
    """
    # Project input features to embedding dimension
    token_proj = nn.Linear(input_dim, embed_dim)
    # Single Transformer layer configuration
    encoder_layer = nn.TransformerEncoderLayer(
        d_model=embed_dim,
        nhead=n_heads,
        dim_feedforward=embed_dim * 4,
        dropout=0.1,
        activation='relu'
    )
    transformer = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)

    # Wrap into a single module
    class TransformerNet(nn.Module):
        def __init__(self):
            super().__init__()
            self.token_proj = token_proj
            self.transformer = transformer

        def forward(self, x):
            # x: (batch_size, input_dim)
            # treat each sample as sequence length=1 for simplicity;
            emb = self.token_proj(x).unsqueeze(0)  # (1, batch, embed_dim)
            out = self.transformer(emb)            # (1, batch, embed_dim)
            return out.squeeze(0)

    return TransformerNet()



In [ ]:

# Client model using transformer
class ClientModel(nn.Module):
    def __init__(self, input_dim, embed_dim, n_heads=2, n_layers=1):
        super().__init__()
        self.encoder = build_transformer(input_dim, embed_dim, n_heads, n_layers)
    def forward(self, x):
        return self.encoder(x)

# Server fusion model also as transformer
class FusionModel(nn.Module):
    def __init__(self, n_clients, embed_dim, n_heads=2, n_layers=1, n_classes=10):
        super().__init__()
        # fusion transformer takes concatenated client embeddings as tokens
        # self.token_proj = nn.Linear(n_clients * embed_dim, embed_dim)
        # so I will be concatenating them all together. They come in the form 
        # each client is (batch_size, seq_len, embed_dim) so I will concatenate them along the second dimension
        # then we'll get (bach_size, seq_len, n_clients * embed_dim)
        layer = nn.TransformerEncoderLayer(d_model=embed_dim,
                                           nhead=n_heads,
                                           dim_feedforward=embed_dim * 4,
                                           dropout=0.1,
                                           activation='relu')
        self.transformer = nn.TransformerEncoder(layer, num_layers=n_layers)
        self.classifier = nn.Linear(embed_dim, n_classes)
    def forward(self, embeddings):
        # embeddings: (batch, n_clients * embed_dim)
        fused = self.token_proj(embeddings).unsqueeze(0)  # (1, batch, embed_dim)
        enc = self.transformer(fused).squeeze(0)         # (batch, embed_dim)
        return self.classifier(enc)



In [ ]:
# ----- Instantiate and train -----
embed_dim = 4
n_clients = len(feature_dims)
n_classes = int(y.max().item() + 1)
clients = [ClientModel(d, embed_dim) for d in feature_dims]
fusion = FusionModel(n_clients, embed_dim, n_heads=2, n_layers=2, n_classes=n_classes)
client_opts = [optim.Adam(c.parameters(), lr=1e-3) for c in clients]
server_opt = optim.Adam(fusion.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()
executor = ThreadPoolExecutor(max_workers=n_clients)

for epoch in range(5):
    total_loss = 0.0
    for *client_feats, labels in loader:
        for opt in client_opts: opt.zero_grad()
        server_opt.zero_grad()
        # compute client embeddings in parallel
        futures = [executor.submit(c, feat) for c, feat in zip(clients, client_feats)]
        embeds = [f.result() for f in futures]
        # concatenate and classify
        concat = torch.cat(embeds, dim=1)
        logits = fusion(concat)
        loss = loss_fn(logits, labels)
        loss.backward()
        server_opt.step()
        for opt in client_opts: opt.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}: avg loss={total_loss/len(loader):.4f}")

print("Training complete on shard:", shard_file)
